In [3]:
from collections import defaultdict, Counter
from functools import partial
import math

In [4]:
def entropy(class_prob):
    return sum(-p * math.log(p,2)
              for p in class_prob
              if p)

In [5]:
def class_prob(labels):
    total_count = len(labels)
    return [count / total_count
           for count in Counter(labels).values()]

In [6]:
def data_entropy(labeled_data):
    labels = [label for _, label in labeled_data]
    probabilities = class_prob(labels)
    return entropy(probabilities)

In [7]:
def partition_entropy(subsets):
    total_count = sum(len(subset) for subset in subsets)
    return sum(data_entropy(subset) * len(subset) / total_count
              for subset in subsets)

In [8]:
inputs = [ 
    ({'level': 'Senior', 'lang': 'Java', 'tweets': 'no', 'phd':'no'}, False),
    ({'level': 'Senior', 'lang': 'Java', 'tweets': 'no', 'phd':'yes'}, False),
    ({'level': 'Mid', 'lang': 'Python', 'tweets': 'no', 'phd':'no'}, True),
    ({'level': 'Junior', 'lang': 'Python', 'tweets': 'no', 'phd':'no'}, True),
    ({'level': 'Junior', 'lang': 'R', 'tweets': 'yes', 'phd':'no'}, True),
    ({'level': 'Junior', 'lang': 'R', 'tweets': 'yes', 'phd':'yes'}, False),
    ({'level': 'Mid', 'lang': 'R', 'tweets': 'yes', 'phd':'yes'}, True),
    ({'level': 'Senior', 'lang': 'Python', 'tweets': 'no', 'phd':'no'}, True),
    ({'level': 'Senior', 'lang': 'R', 'tweets': 'yes', 'phd':'yes'}, True),
    ({'level': 'Junior', 'lang': 'Python', 'tweets': 'yes', 'phd':'no'}, True),
    ({'level': 'Senior', 'lang': 'Python', 'tweets': 'yes', 'phd':'yes'}, True),
    ({'level': 'Mid', 'lang': 'Python', 'tweets': 'no', 'phd':'yes'}, True),
    ({'level': 'Mid', 'lang': 'Java', 'tweets': 'yes', 'phd':'no'}, True),
    ({'level': 'Junior', 'lang': 'Python', 'tweets': 'no', 'phd':'yes'}, True),
]

In [9]:
def partition_by(inputs, attribute):
    groups = defaultdict(list)
    for input in inputs:
        key = input[0][attribute]
        groups[key].append(input)
    return groups

In [10]:
def partition_entropy_by(inputs, attribute):
    partitions = partition_by(inputs, attribute)
    return partition_entropy(partitions.values())

In [11]:
for key in ['level', 'lang', 'tweets', 'phd']:
    print(key, partition_entropy_by(inputs, key))

level 0.6045995319078682
lang 0.4285714285714286
tweets 0.7273966735744792
phd 0.7273966735744792


In [12]:
senior_inputs = [(input, label)
                for input, label in inputs if input["level"] == "Senior"]
for key in ['lang', 'tweets', 'phd']:
    print(key, partition_entropy_by(senior_inputs, key))

lang 0.0
tweets 0.5509775004326938
phd 0.9509775004326938


In [13]:
def classify(tree, input):
    if tree in [True, False]:
        return tree
    
    attribute, subtree_dict = tree
    
    subtree_key = input.get(attribute)
    
    if subtree_key not in subtree_dict:
        subtree_key = None
        
    subtree = subtree_dict[subtree_key]
    return classify(subtree, input)

In [14]:
def build_tree_id3(inputs, split_candidates=None):
    if split_candidates is None:
        split_candidates = inputs[0][0].keys()
        
    num_inputs = len(inputs)
    num_trues = len([label for item, label in inputs if label])
    num_falses = num_inputs - num_trues
    
    if num_trues == 0: return False
    if num_falses ==0: return True
    
    if not split_candidates:
        return num_trues >= num_falses
    
    best_attribute = min(split_candidates, 
                         key=partial(partition_entropy_by, inputs))
    
    partitions = partition_by(inputs, best_attribute)
    new_candidiates = [a for a in split_candidates
                      if a != best_attribute]
    
    subtrees = {attribute: build_tree_id3(subset, new_candidiates)
               for attribute, subset in partitions.items()}
    
    subtrees[None] = num_trues > num_falses
    
    return (best_attribute, subtrees)

In [15]:
tree = build_tree_id3(inputs)

In [49]:
classify(tree, {"level": "Junior",
               "lang": "Java",
               "tweets": "yes",
               "phd": "no"})

False

In [16]:
classify(tree, {"level": "Senior"})

True

In [17]:
inputs_20 = [
    ({'level': 'Senior', 'lang': 'Java', 'tweets': 'no', 'phd':'no'}, False),
    ({'level': 'Senior', 'lang': 'Java', 'tweets': 'no', 'phd':'yes'}, False),
    ({'level': 'Mid', 'lang': 'Python', 'tweets': 'no', 'phd':'no'}, True),
    ({'level': 'Junior', 'lang': 'Python', 'tweets': 'no', 'phd':'no'}, True),
    ({'level': 'Junior', 'lang': 'R', 'tweets': 'yes', 'phd':'no'}, True),
    ({'level': 'Junior', 'lang': 'R', 'tweets': 'yes', 'phd':'yes'}, False),
    ({'level': 'Mid', 'lang': 'R', 'tweets': 'yes', 'phd':'yes'}, True),
    ({'level': 'Senior', 'lang': 'Python', 'tweets': 'no', 'phd':'no'}, True),
    ({'level': 'Senior', 'lang': 'R', 'tweets': 'yes', 'phd':'yes'}, True),
    ({'level': 'Junior', 'lang': 'Python', 'tweets': 'yes', 'phd':'no'}, True),
    ({'level': 'Senior', 'lang': 'Python', 'tweets': 'yes', 'phd':'yes'}, True),
    ({'level': 'Mid', 'lang': 'Python', 'tweets': 'no', 'phd':'yes'}, True),
    ({'level': 'Mid', 'lang': 'Java', 'tweets': 'yes', 'phd':'no'}, True),
    ({'level': 'Junior', 'lang': 'Python', 'tweets': 'no', 'phd':'yes'}, True),
    ({'level': 'Senior', 'lang': 'Java', 'tweets': 'yes', 'phd':'yes'}, True),
    ({'level': 'Junior', 'lang': 'Java', 'tweets': 'no', 'phd':'no'}, False),
    ({'level': 'Mid', 'lang': 'Java', 'tweets': 'yes', 'phd':'no'}, False),
    ({'level': 'Senior', 'lang': 'Python', 'tweets': 'no', 'phd':'yes'}, True),
    ({'level': 'Intern', 'lang': 'R', 'tweets': 'no', 'phd':'no'}, False),
    ({'level': 'Senior', 'lang': 'Java', 'tweets': 'yes', 'phd':'no'}, False),
    ({'level': 'Senior', 'lang': 'Python', 'tweets': 'no', 'phd':'no'}, True),
    ({'level': 'Junior', 'lang': 'JavaScript', 'tweets': 'no', 'phd':'no'}, False),
    ({'level': 'Mid', 'lang': 'R', 'tweets': 'no', 'phd':'yes'}, False),
    ({'level': 'Intern', 'lang': 'JavaScript', 'tweets': 'yes', 'phd':'no'}, False),
    ({'level': 'Intern', 'lang': 'Python', 'tweets': 'no', 'phd':'no'}, True),
    ({'level': 'Mid', 'lang': 'Java', 'tweets': 'no', 'phd':'yes'}, True),
    ({'level': 'Junior', 'lang': 'Java', 'tweets': 'yes', 'phd':'no'}, False),
    ({'level': 'Senior', 'lang': 'R', 'tweets': 'no', 'phd':'no'}, True),
    ({'level': 'Senior', 'lang': 'JavaScript', 'tweets': 'yes', 'phd':'yes'}, True),
    ({'level': 'Mid', 'lang': 'Python', 'tweets': 'no', 'phd':'no'}, False),
    ({'level': 'Junior', 'lang': 'Java', 'tweets': 'no', 'phd':'no'}, False),
    ({'level': 'Senior', 'lang': 'R', 'tweets': 'yes', 'phd':'yes'}, False),
    ({'level': 'Mid', 'lang': 'Python', 'tweets': 'no', 'phd':'no'}, True),
    ({'level': 'Intern', 'lang': 'R', 'tweets': 'yes', 'phd':'no'}, True)

]

In [31]:
tree20 = build_tree_id3(inputs_20)
classify(tree20, {"level": "Senior"})

True

In [32]:
inputs_50 = [
    ({'level': 'Senior', 'lang': 'Java', 'tweets': 'no', 'phd':'no'}, False),
    ({'level': 'Senior', 'lang': 'Java', 'tweets': 'no', 'phd':'yes'}, False),
    ({'level': 'Mid', 'lang': 'Python', 'tweets': 'no', 'phd':'no'}, True),
    ({'level': 'Junior', 'lang': 'Python', 'tweets': 'no', 'phd':'no'}, True),
    ({'level': 'Junior', 'lang': 'R', 'tweets': 'yes', 'phd':'no'}, True),
    ({'level': 'Junior', 'lang': 'R', 'tweets': 'yes', 'phd':'yes'}, False),
    ({'level': 'Mid', 'lang': 'R', 'tweets': 'yes', 'phd':'yes'}, True),
    ({'level': 'Senior', 'lang': 'Python', 'tweets': 'no', 'phd':'no'}, True),
    ({'level': 'Senior', 'lang': 'R', 'tweets': 'yes', 'phd':'yes'}, True),
    ({'level': 'Junior', 'lang': 'Python', 'tweets': 'yes', 'phd':'no'}, True),
    ({'level': 'Senior', 'lang': 'Python', 'tweets': 'yes', 'phd':'yes'}, True),
    ({'level': 'Mid', 'lang': 'Python', 'tweets': 'no', 'phd':'yes'}, True),
    ({'level': 'Mid', 'lang': 'Java', 'tweets': 'yes', 'phd':'no'}, True),
    ({'level': 'Junior', 'lang': 'Python', 'tweets': 'no', 'phd':'yes'}, True),
    ({'level': 'Senior', 'lang': 'Java', 'tweets': 'yes', 'phd':'yes'}, True),
    ({'level': 'Junior', 'lang': 'Java', 'tweets': 'no', 'phd':'no'}, False),
    ({'level': 'Mid', 'lang': 'Java', 'tweets': 'yes', 'phd':'no'}, False),
    ({'level': 'Senior', 'lang': 'Python', 'tweets': 'no', 'phd':'yes'}, True),
    ({'level': 'Intern', 'lang': 'R', 'tweets': 'no', 'phd':'no'}, False),
    ({'level': 'Senior', 'lang': 'Java', 'tweets': 'yes', 'phd':'no'}, False),
    ({'level': 'Senior', 'lang': 'Python', 'tweets': 'no', 'phd':'no'}, True),
    ({'level': 'Junior', 'lang': 'JavaScript', 'tweets': 'no', 'phd':'no'}, False),
    ({'level': 'Mid', 'lang': 'R', 'tweets': 'no', 'phd':'yes'}, False),
    ({'level': 'Intern', 'lang': 'JavaScript', 'tweets': 'yes', 'phd':'no'}, False),
    ({'level': 'Intern', 'lang': 'Python', 'tweets': 'no', 'phd':'no'}, True),
    ({'level': 'Mid', 'lang': 'Java', 'tweets': 'no', 'phd':'yes'}, True),
    ({'level': 'Junior', 'lang': 'Java', 'tweets': 'yes', 'phd':'no'}, False),
    ({'level': 'Senior', 'lang': 'R', 'tweets': 'no', 'phd':'no'}, True),
    ({'level': 'Senior', 'lang': 'JavaScript', 'tweets': 'yes', 'phd':'yes'}, True),
    ({'level': 'Mid', 'lang': 'Python', 'tweets': 'no', 'phd':'no'}, False),
    ({'level': 'Junior', 'lang': 'Java', 'tweets': 'no', 'phd':'no'}, False),
    ({'level': 'Senior', 'lang': 'R', 'tweets': 'yes', 'phd':'yes'}, False),
    ({'level': 'Mid', 'lang': 'Python', 'tweets': 'no', 'phd':'no'}, True),
    ({'level': 'Mid', 'lang': 'R', 'tweets': 'yes', 'phd':'no'}, True),
    ({'level': 'Intern', 'lang': 'R', 'tweets': 'yes', 'phd':'no'}, True),
    ({'level': 'Senior', 'lang': 'Python', 'tweets': 'yes', 'phd':'no'}, False),
    ({'level': 'Mid', 'lang': 'Python', 'tweets': 'yes', 'phd':'no'}, True),
    ({'level': 'Senior', 'lang': 'Python', 'tweets': 'yes', 'phd':'no'}, True),
    ({'level': 'Junior', 'lang': 'JavaScript', 'tweets': 'yes', 'phd':'no'}, False),
    ({'level': 'Junior', 'lang': 'R', 'tweets': 'yes', 'phd':'no'}, True),
    ({'level': 'Intern', 'lang': 'Python', 'tweets': 'yes', 'phd':'no'}, False),
    ({'level': 'Senior', 'lang': 'Python', 'tweets': 'yes', 'phd':'no'}, True),
    ({'level': 'Mid', 'lang': 'R', 'tweets': 'yes', 'phd':'yes'}, True),
    ({'level': 'Mid', 'lang': 'R', 'tweets': 'no', 'phd':'no'}, False),
    ({'level': 'Junior', 'lang': 'R', 'tweets': 'yes', 'phd':'yes'}, True),
    ({'level': 'Intern', 'lang': 'R', 'tweets': 'yes', 'phd':'no'}, False),
    ({'level': 'Intern', 'lang': 'Java', 'tweets': 'no', 'phd':'no'}, True),
    ({'level': 'Senior', 'lang': 'R', 'tweets': 'yes', 'phd':'yes'}, False),
    ({'level': 'Senior', 'lang': 'Java', 'tweets': 'no', 'phd':'no'}, False),
    ({'level': 'Senior', 'lang': 'Python', 'tweets': 'no', 'phd':'no'}, True),
    ({'level': 'Intern', 'lang': 'R', 'tweets': 'no', 'phd':'no'}, True),
    ({'level': 'Senior', 'lang': 'Java', 'tweets': 'yes', 'phd':'yes'}, True),
    ({'level': 'Intern', 'lang': 'JavaScript', 'tweets': 'yes', 'phd':'no'}, True),
    ({'level': 'Intern', 'lang': 'Python', 'tweets': 'yes', 'phd':'no'}, True),
    ({'level': 'Senior', 'lang': 'R', 'tweets': 'no', 'phd':'no'}, True),
    ({'level': 'Mid', 'lang': 'R', 'tweets': 'yes', 'phd':'no'}, True),
    ({'level': 'Mid', 'lang': 'Java', 'tweets': 'no', 'phd':'yes'}, True),
    ({'level': 'Senior', 'lang': 'Python', 'tweets': 'yes', 'phd':'no'}, True),
    ({'level': 'Junior', 'lang': 'JavaScript', 'tweets': 'no', 'phd':'no'}, False),
    ({'level': 'Junior', 'lang': 'Java', 'tweets': 'yes', 'phd':'no'}, True),
    ({'level': 'Intern', 'lang': 'Python', 'tweets': 'yes', 'phd':'no'}, False),
    ({'level': 'Senior', 'lang': 'Python', 'tweets': 'no', 'phd':'yes'}, True),
    ({'level': 'Intern', 'lang': 'Python', 'tweets': 'no', 'phd':'no'}, False),
    ({'level': 'Junior', 'lang': 'Python', 'tweets': 'yes', 'phd':'yes'}, False)
    
]

In [36]:
tree50 = build_tree_id3(inputs_50)
classify(tree50, {"level": "Senior"})

True